In [499]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [500]:
url = "https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query=춘천+날씨"
r = requests.get(url)
html = BeautifulSoup(r.text, 'html.parser')

temp_info = html.select('p.info_temperature')

# 현재 기온
temp_info[0].select_one('span.todaytemp').text

'26'

# 3일 기온

In [501]:
# 업데이트 정보창
time_info = html.select('p.guide')

# 크롤링 시 업데이트 시간(hour) 정보
update_time = time_info[0].select_one('span.update').text
update_time_int = int(time_info[0].select_one('span.update').text[:2])

Temperature = []

# 기상정보 업데이트 시간 오전 / 오후 구분
if (update_time_int < 12)==True :
    for i in range(len(temp_info)):
        Temperature.append(temp_info[i].select_one('span.todaytemp').text)
else:
    for i in range(len(temp_info)):
        Temperature.append(temp_info[i].select_one('span.todaytemp').text)
        
#df_temp = pd.DataFrame(Temperature, columns=['기온'])
# 오전에 검색했을 때 당일 오후 기온 어떻게 표시되는지 확인 필요
# DataFrame 변환 시 컬럼 개수 통일 위해서
# (오늘- 오후), 내일, 모레 - 오전/오후   columns 5개
# (오늘, 내일, 모레 - 오전/오후)    columns 6개

In [502]:
Temperature

['26', '20', '29', '18', '31']

In [503]:
df_temp = pd.DataFrame(Temperature, columns=['기온'])                                                  
df_temp['기준시각'] = update_time
df_temp

,기온,기준시각
0,26,17:30
1,20,17:30
2,29,17:30
3,18,17:30
4,31,17:30


### span 갯수가 기온 3개, 나머지 강수, 바람, 습도는 2개씩 순환함

In [504]:
WeatherByTime[0].select('li > dl > dd.weather_item._dotWrapper > span')[:7]

[<span>26</span>,
 <span class="blind">도</span>,
 <span class="dot_point today _dot"></span>,
 <span>26</span>,
 <span class="blind">도</span>,
 <span class="dot_point today _dot"></span>,
 <span>25</span>]

In [505]:
WeatherByTime[3].select('li > dl > dd.weather_item._dotWrapper > span')[:7]

[<span>61</span>,
 <span class="dot_point today _dot"></span>,
 <span>55</span>,
 <span class="dot_point today _dot"></span>,
 <span>65</span>,
 <span class="dot_point today _dot"></span>,
 <span>65</span>]

## 5시간 뒤까지의 시간별 기온, 강수, 바람, 습도 정보를 담은 2차원 리스트 생성

In [506]:
# 시간대별 날씨 카테고리에 기온, 강수, 바람, 습도에 순차적으로 접근하기 위해서
# 상위 카테고리로 접근
WeatherByTime = html.select('ul.list_area')

# 향후 4시간 동안의 기온, 강수, 바람, 습도 리스트 생성
weather_condition = [[],[],[],[]]

# weather_condition[i] (순서대로 기온, 강수, 바람, 습도 리스트)에 넣기 위해서 i 지정 후 for 문 실행
for i in range(len(weather_condition)):
    
    # 밑에 문장의 결과처럼 기온 span이 3개씩, 그 외에는 span이 2개씩 순환
    # i==0(기온)일 때는 3개씩 순환, 나머지는 2개씩 순환하므로 range 지정
    if i==0:
        for j in range(0,15,3):
            weather_condition[i].append((WeatherByTime[i].select
                                      ('li > dl > dd.weather_item._dotWrapper > span')[j].text))
    else:
        for j in range(0,10,2):
            weather_condition[i].append((WeatherByTime[i].select
                                      ('li > dl > dd.weather_item._dotWrapper > span')[j].text))

# 현재 기준시각으로부터 5시간 뒤의 시간 리스트 생성 후 WeatherByTime의 첫 번째 리스트로 삽입
time_5 = []
for i in range(5):
    time_5.append(WeatherByTime[0].select('li > dl > dd.item_time > span')[i].text.strip())                               
weather_condition.insert(0,time_5)


weather_condition

[['17시', '18시', '19시', '20시', '21시'],
 ['26', '26', '25', '25', '24'],
 ['30', '30', '30', '30', '60'],
 ['1', '2', '2', '1', '0'],
 ['61', '55', '65', '65', '70']]

In [507]:
col_name_1 = ['Temp', 'Precip(%)','WindSpd(m/s)','Humidity(%)']
weather_5h = pd.DataFrame({col_name[0]:weather_condition[1],
                  col_name[1]:weather_condition[2],
                  col_name[2]:weather_condition[3],
                  col_name[3]:weather_condition[4]},
                  index=weather_condition[0])
weather_5h


,Temp,Precip(%),WindSpd(m/s),Humidity(%)
17시,26,30,1,61
18시,26,30,2,55
19시,25,30,2,65
20시,25,30,1,65
21시,24,60,0,70


# FineDust / Fine Particulate matter / Ozone

In [508]:
etc_info = html.select('dl > dd.lv2 > span.num')
etc_info

[<span class="num">42㎍/㎥</span>,
 <span class="num">30㎍/㎥</span>,
 <span class="num">0.072ppm</span>]

In [514]:
etc_list = []
for i in range(3):
    etc_list.append(etc_info[i].text)
etc_list

['42㎍/㎥', '30㎍/㎥', '0.072ppm']

In [515]:
etc_list[0]

'42㎍/㎥'